In [ ]:
#--------------------( Importar bibliotecas )--------------------#

import pandas as pd
import pyodbc
import pymssql
import datetime
import smtplib
from smtplib import SMTP
import json
import re
import win32com.client as win32
import schedule
import time
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")



In [ ]:
##--------------------( Configurar o modo como eu vejo as tabelas )--------------------#

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

Conectar no SQL Server

In [ ]:
def conectar_sql():
    server = 'tms-drp-oficial.database.windows.net'
    database = 'tms'
    user = 'appTableau'
    password = 'Tjd82#jks2'

    tentativas = 3

    for tentativa in range(1, tentativas + 1):
        try:
            conexao = pymssql.connect(server=server, database=database, user=user, password=password)
            print(f"Conexão bem-sucedida na tentativa {tentativa}")
            return conexao
        except Exception as e:
            print(f"Tentativa {tentativa} falhou. Erro: {e}")
            if tentativa == tentativas:
                raise

    # conexao = pymssql.connect(server=server, database=database, user=user, password=password)

    # return conexao

Criar base de dados usando a query

In [ ]:
def gerar_query():

    # Query - Aqui pode modificar se precisar
    query = """
    SELECT DISTINCT
    ASI_NUMERO_ASSISTENCIA AS 'Assistencia',
    OSV_ID 'OS',
    TEO_NOME 'statusAcaoNome',
    TEO_ID,
    OSV_DATA_ABERTURA 'dataAberturaOS',
    OSV_DATA_CONCLUSAO 'ConclusaoServico',
    EOS_DATA_INICIO 'dataInicio',
    EOS_DATA_FIM 'dataFim',
    SOS_NOME 'statusOS',
    MSP_SLA_LIMITE_NEGOCIADO,
    (EOS_DATA_FIM - EOS_DATA_INICIO) tempoDuracao,
    (SELECT DATEDIFF (MINUTE, EOS_DATA_INICIO, EOS_DATA_FIM)) duracaoMinutos,
    (SELECT DATEADD (MINUTE, MSP_SLA_LIMITE_NEGOCIADO, EOS_DATA_INICIO)) tempoFimPrevia,
    --FORMAT(SWITCHOFFSET(GETDATE(), '-03:00'), 'yyyy-MM-dd HH:mm:ss') dataAtual
    TSE_NOME 'tipoServiço',
    OSL_MUNICIPIO Cidade,
    OSL_UF Estado,
    TEV_SEG_ID segmento

    FROM ORDEM_SERVICO

    LEFT JOIN ETAPA_ORDEM_SERVICO ON EOS_OSV_ID = OSV_ID
    LEFT JOIN TIPO_ETAPA_ORDEM_SERVICO ON TEO_ID = EOS_TEO_ID
    LEFT JOIN STATUS_ORDEM_SERVICO ON SOS_ID = OSV_SOS_ID
    JOIN MISSAO ON MSO_OSV_ID = OSV_ID
    JOIN MISSAO_PRESTADOR ON MSP_MSO_ID = MSO_ID
    JOIN ACIONAMENTO_PRESTADOR ON APR_MSP_MSO_ID = MSO_ID
    JOIN PRESTADOR ON APR_PRS_ID = PRS_ID
    LEFT JOIN TIPO_SERVICO ON TSE_ID = OSV_TSE_ID
    LEFT JOIN ORDEM_SERVICO_LOCALIDADE ON OSL_OSV_ID = OSV_ID
    LEFT JOIN ASSISTENCIA ON ASI_ID = OSV_ASI_ID
    LEFT JOIN TIPO_EVENTO ON TEV_ID = ASI_TEV_ID

    WHERE CONVERT(DATE, OSV_DATA_ABERTURA) = CONVERT(DATE, GETDATE())
    AND SOS_NOME <> 'Cancelado' 
    AND SOS_NOME <>  'Agendado' 
    AND SOS_NOME <> 'Concluido' 
    AND SOS_NOME <> 'Realizando entrega'
    AND TEO_ID <> 4
    AND TEO_ID <> 2
    AND OSV_DATA_AGENDAMENTO IS NULL
    AND TEV_SEG_ID = 1
    AND OSL_NUMERO_SEQUENCIA = 1

    ORDER BY OSV_ID, TEO_ID
    """

    # Fazer a conexão e olhar os dados
    conexao = conectar_sql()
    df = pd.read_sql_query(query, conexao)
    df1 = df

    conexao.close()
    
    return df, df1

Tempo de chegada

In [ ]:
def tempoChegada(df):

    # Criar coluna com data e hora atual
    df['horaAtual'] = datetime.datetime.now()

    # Formatar tudo para Datetime
    colunas = ['dataAberturaOS','dataInicio','dataFim','horaAtual','tempoDuracao','tempoFimPrevia']

    df[colunas] = df[colunas].apply(pd.to_datetime)

    # Criar tempo de Experiência até o momento
    df['tempoExperienciaAgora'] = df['horaAtual'] - df['dataAberturaOS']

    # Tranformar em modelo datetime
    tempo_timedelta = pd.to_timedelta(df['tempoExperienciaAgora'])
    data_referencia = pd.Timestamp.now().normalize()
    tempo_datetime = data_referencia + tempo_timedelta

    # Criar coluna resultado sendo ele como tempo que finaliza a prévia que foi informada ao cliente
    filtro = (df['statusAcaoNome'] == 'Chegada') & (df['dataFim'].isnull())
    df['ResultadoTempoChegada'] = ''
    df.loc[filtro, 'ResultadoTempoChegada'] = (df.loc[filtro, 'horaAtual'].apply(lambda x: x.timestamp()) - df.loc[filtro, 'tempoFimPrevia'].apply(lambda x: x.timestamp())) / 60

    # Criar um campo com tempo de atendimento ultrapassado, ou seja, vai mostrar em minutos, quanto tempos estamos a partir do final do acionamento
    x = df['ResultadoTempoChegada']
    def convert_to_hms(x):
        if x:
            hours = int(x // 3600)
            minutes = int((x % 3600) // 60)
            seconds = int(x % 60)
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        else:
            return ""

    # Lançar informações nos campos que estão em branco, pois sem dados, não consigo fazer contas
    df['ResultadoTempoChegada'] = pd.to_numeric(df['ResultadoTempoChegada'], errors='coerce')

    # Criar o campo status que vai mostrar se estamos dentro da prévia ou fora
    df['Status'] = df['ResultadoTempoChegada'].apply(lambda x: 'foraPrevia' if x > 0 else 'Finalizado')

    def formatar_tempo(tempo_minutos):
        horas = tempo_minutos // 60
        minutos = tempo_minutos % 60
        segundos = int((tempo_minutos - int(tempo_minutos)) * 60)
        return '{:02d}:{:02d}:{:02d}'.format(int(horas), int(minutos), segundos)

    # Transformar o campo resultado que estava em minutos no modelo de dia, hora, minutos e segundos
 
    df['tempoForaPrevia'] = np.where(df['ResultadoTempoChegada'] > 0, df['ResultadoTempoChegada'].apply(lambda x: str(pd.to_timedelta(x, unit='m'))), 'dentroPrevia')

    # Tranformar a coluna tempoForaPrevia em datetime, conforme os demais campos.
    df['tempoForaPrevia'] = df['tempoForaPrevia'].apply(lambda x: pd.Timestamp.now().normalize() + x if isinstance(x, pd.Timedelta) else x)

    ## Trocar valores em branco, com não finalizado
    df['dataFim'].fillna(value='Em andamento', inplace=True)

    ## Desejo visualizar os serviços que estão em andamento. Vou excluir todos que já foram finalizados
    tabela = df.loc[df['dataFim'] == 'Em andamento']

    # Serviços com status somente Chegada
    tabelaChegada = tabela.loc[tabela['statusAcaoNome'] == 'Chegada']

    # Tabela Chegada
    tabelaChegadaFinal = tabelaChegada[['Assistencia','OS','dataInicio','MSP_SLA_LIMITE_NEGOCIADO','tempoFimPrevia','Status','duracaoMinutos','tempoExperienciaAgora', 'horaAtual','tipoServiço', 'Cidade', 'Estado']]

    tabelaChegadaFinal = tabelaChegadaFinal.loc[tabela['Status'] == 'foraPrevia']

    tabelaChegadaFinal['tempoExcedido'] = tabelaChegadaFinal['horaAtual'] - tabelaChegadaFinal['tempoFimPrevia']

    # Tranformar em modelo datetime - Tempo fora da prévia
    tempo_timedelta = pd.to_timedelta(tabelaChegadaFinal['tempoExcedido'])
    data_referencia = pd.Timestamp.now().normalize()
    tempo_datetime = data_referencia + tempo_timedelta

    # Criar tempo excedido depois da prévia que o prestador informou e deixar como número inteiro (minutos) 
    tabelaChegadaFinal['tempoExcedido'] = round(tabelaChegadaFinal['tempoExcedido'].dt.total_seconds() / 60).astype(int)

    # Renomear a tabela do SLA
    tabelaChegadaFinal = tabelaChegadaFinal.rename(columns={'MSP_SLA_LIMITE_NEGOCIADO': 'SLA'})

    # Criar uma coluna somente com a hora do inicio do acionamento
    tabelaChegadaFinal['horaInicio'] = tabelaChegadaFinal['dataInicio']

    tabelaChegadaFinal['ultimaAtualizacao'] = tabelaChegadaFinal['horaAtual']

    # Prazo para finalizar a prévia informada pelo prestador
    tabelaChegadaFinal['ultimaAtualizacao'] = pd.to_datetime(tabelaChegadaFinal['ultimaAtualizacao'])

    # Tranformei o tempo exedido que estava em minutos para visualização de horas
    tabelaChegadaFinal['tempoForaPrevia'] = pd.to_datetime(tabelaChegadaFinal['tempoExcedido'], unit='m').dt.strftime('%H:%M:%S')

    ## Transformar todas as colunas que contenha data, hora e modelo em somente hora
    # Data Inicio
    tabelaChegadaFinal['dataInicio'] = pd.to_datetime(tabelaChegadaFinal['dataInicio']) 

    lista = []
    for i in tabelaChegadaFinal['dataInicio']:
        x = i.strftime("%d/%m")
        lista.append(x)

    tabelaChegadaFinal['dataInicio'] = lista

    colunas = ['horaInicio', 'tempoFimPrevia', 'ultimaAtualizacao']
    tabelaChegadaFinal[colunas] = tabelaChegadaFinal[colunas].apply(lambda x: x.dt.strftime("%H:%M:%S"))

    # Tranformar a coluna tempo de experiência em datetime
    tabelaChegadaFinal['tempoExperienciaAgora'] = tabelaChegadaFinal['tempoExperienciaAgora'].apply(lambda x: pd.Timestamp.now().normalize() + x if isinstance(x, pd.Timedelta) else x)

    # Converter para somente hora
    lista = []
    for i in tabelaChegadaFinal['tempoExperienciaAgora']:
        x = i.strftime("%H:%M:%S")
        lista.append(x)

    tabelaChegadaFinal['tempoExperienciaAgora'] = lista

    # Serviços de 30 à 60 acima do tempo de chegada - Avisar o Coordenador
    tabelaChegada1 = tabelaChegadaFinal.query('30 < tempoExcedido <= 60')[['Assistencia','OS', 'dataInicio', 'horaInicio', 'SLA', 'tempoFimPrevia', 'ultimaAtualizacao', 'tempoForaPrevia', 'tempoExperienciaAgora', 'tipoServiço', 'Cidade', 'Estado']]

    # Serviços de 60 à 90 minutos em acionamento - Avisar o Gerente
    tabelaChegada2 = tabelaChegadaFinal.query('60 < tempoExcedido <= 90')[['Assistencia','OS', 'dataInicio', 'horaInicio', 'SLA', 'tempoFimPrevia', 'ultimaAtualizacao', 'tempoForaPrevia', 'tempoExperienciaAgora', 'tipoServiço', 'Cidade', 'Estado']]

    # Serviços acima de 90 minutos - Avisar o Superintendente
    tabelaChegada3 = tabelaChegadaFinal.query('tempoExcedido > 90')[['Assistencia','OS', 'dataInicio', 'horaInicio', 'SLA', 'tempoFimPrevia', 'ultimaAtualizacao', 'tempoForaPrevia', 'tempoExperienciaAgora', 'tipoServiço', 'Cidade', 'Estado']]

    # Crie um dicionário com as opções e os resultados
    resultadoChegada = {
        'Entre 30 e 60 minutos': tabelaChegada1['OS'].count(),
        'Entre 60 e 90 minutos': tabelaChegada2['OS'].count(),
        'Acima de 90 minutos': tabelaChegada3['OS'].count()
    }

    # Crie a tabela a partir do dicionário
    resultadoChegada = pd.DataFrame.from_dict(resultadoChegada, orient='index', columns=['Quantidade'])

    # Inserir nome na coluna Index
    resultadoChegada.rename_axis('Range',axis=0, inplace = True)

    return resultadoChegada, tabelaChegada1, tabelaChegada2, tabelaChegada3


Em acionamento

In [ ]:
df, df1 = gerar_query()

In [ ]:
def tempoAcionamento(df1):

    df = df1
    # Criar coluna com data e hora atual
    df['horaAtual'] = datetime.datetime.now()

    # Formatar tudo para Datetime
    colunas = ['dataAberturaOS','dataInicio','dataFim','horaAtual','tempoDuracao','tempoFimPrevia' ]

    df[colunas] = df[colunas].apply(pd.to_datetime, errors='coerce')

    # Criar tempo de Experiência até o momento
    df['tempoExperienciaAgora'] = df['horaAtual'] - df['dataAberturaOS']

    # Tranformar em modelo datetime
    tempo_timedelta = pd.to_timedelta(df['tempoExperienciaAgora'])
    data_referencia = pd.Timestamp.now().normalize()
    tempo_datetime = data_referencia + tempo_timedelta

    # Filtrar a tabela somente com serviços com Status Em acionamento e Data Fim em branco
    df = df.loc[(df['statusAcaoNome'] == 'Em Acionamento') & (df['dataFim'].isnull())]

    # Criar coluna com data e hora atual
    df['horaAtual'] = datetime.datetime.now()
    df['horaAtual'] = pd.to_datetime(df['horaAtual'])

    # Tratar a data inicio e deixar em formato para fazer calculo
    df['dataInicio'] = pd.to_datetime(df['dataInicio'])

    # Tratar a data fim
    df['dataFim'] = pd.to_datetime(df['dataFim'])

    ## Trocar valores em branco, com não finalizado
    df['dataFim'].fillna(value='Em andamento', inplace=True)

    # Função para calcular a duração
    def calcular_duracao(row):
        if row['statusAcaoNome'] == 'Em Acionamento' and row['dataFim'] == 'Em andamento':
            return row['horaAtual'] - row['dataInicio']
        else:
            return np.nan

    # Criar coluna com duração do acionamento
    df['duracaoAcionamento'] = df.apply(calcular_duracao, axis=1)

    # Tranformar a coluna Duração Acionamento em datetime
    df['duracaoAcionamento'] = pd.to_datetime('1970-01-01') + df['duracaoAcionamento']

    # Tabela em acionamento
    tabelaEmAcionamentoFinal = df[['Assistencia','OS', 'dataInicio', 'duracaoAcionamento', 'tipoServiço', 'Cidade', 'Estado']]

    tabelaEmAcionamentoFinal['duracaoMinutos'] = tabelaEmAcionamentoFinal['duracaoAcionamento'].apply(lambda x: int(x.timestamp()))

    # Criar uma coluna somente com a hora do inicio do acionamento
    tabelaEmAcionamentoFinal['horaInicio'] = tabelaEmAcionamentoFinal['dataInicio']

    ## Transformar todas as colunas que contenha data, hora e modelo em somente hora

    # Data Inicio
    tabelaEmAcionamentoFinal['dataInicio'] = pd.to_datetime(tabelaEmAcionamentoFinal['dataInicio']) 

    lista = []
    for i in tabelaEmAcionamentoFinal['dataInicio']:
        x = i.strftime("%d/%m")
        lista.append(x)

    tabelaEmAcionamentoFinal['dataInicio'] = lista

    colunas = ['horaInicio', 'duracaoAcionamento']
    tabelaEmAcionamentoFinal[colunas] = tabelaEmAcionamentoFinal[colunas].apply(lambda x: x.dt.strftime("%H:%M:%S"))

    # Serviços de 30 à 60 minutos em acionamento - Avisar o Coordenador
    tabelaAcionamento1 = tabelaEmAcionamentoFinal.query('1800 < duracaoMinutos <= 3600')[['Assistencia','OS', 'dataInicio', 'horaInicio', 'duracaoAcionamento', 'tipoServiço', 'Cidade', 'Estado']]

    # Serviços de 60 à 90 minutos em acionamento - Avisar o Gerente
    tabelaAcionamento2 = tabelaEmAcionamentoFinal.query('3600 < duracaoMinutos <= 5400')[['Assistencia','OS', 'dataInicio', 'horaInicio', 'duracaoAcionamento', 'tipoServiço', 'Cidade', 'Estado']]

    # Serviços acima de 90 minutos - Avisar o Superintendente
    tabelaAcionamento3 = tabelaEmAcionamentoFinal.query('duracaoMinutos > 5400')[['Assistencia','OS', 'dataInicio', 'horaInicio', 'duracaoAcionamento', 'tipoServiço', 'Cidade', 'Estado']]

    # Tabela de acionamento
    # Crie um dicionário com as opções e os resultados
    resultadosAcionamento = {
        'Entre 30 e 60 minutos': tabelaAcionamento1['OS'].count(),
        'Entre 60 e 90 minutos': tabelaAcionamento2['OS'].count(),
        'Acima de 90 minutos': tabelaAcionamento3['OS'].count()
    }

    # Crie a tabela a partir do dicionário
    resultadosAcionamento = pd.DataFrame.from_dict(resultadosAcionamento, orient='index', columns=['Quantidade'])

    # Inserir nome na coluna Index
    resultadosAcionamento.rename_axis('Range',axis=0, inplace = True)

    return resultadosAcionamento, tabelaAcionamento1, tabelaAcionamento2, tabelaAcionamento3


Experiência do Cliente

In [ ]:
def tempoExperiencia():

    # Editar a query conforme eu precisar. Ideal inserir somente os campos que vou usar

    queryExperiencia = """
    SELECT DISTINCT
    ASI_NUMERO_ASSISTENCIA AS 'Assistência',
    OSV_ID 'Ordem Serviço',
    OSV_DATA_ABERTURA 'Data abertura OS',
    EOS_DATA_INICIO 'Inicio acionamento',
    EOS_DATA_FIM 'Data Fim',
    OSV_DATA_AGENDAMENTO 'Data agendamento',
    SOS_NOME 'Status do serviço',
    EOS_TEO_ID,
    TEO_NOME,
    TSE_NOME,
    SEG_NOME,
    PRS_NOME,
    ASI_SEGURADO,
    TAC_NOME

    FROM ORDEM_SERVICO
    LEFT JOIN ETAPA_ORDEM_SERVICO ON EOS_OSV_ID = OSV_ID
    LEFT JOIN STATUS_ORDEM_SERVICO ON SOS_ID = OSV_SOS_ID
    LEFT JOIN TIPO_ETAPA_ORDEM_SERVICO ON TEO_ID = EOS_TEO_ID
    LEFT JOIN TIPO_SERVICO ON OSV_TSE_ID = TSE_ID
    LEFT JOIN PARAMETRO_MOTOR ON PMT_ID = OSV_PMT_ID
    LEFT JOIN PARAMETRO_MOTOR_FILTRO ON PMF_PMT_ID = PMT_ID
    LEFT JOIN PARAMETRO_MOTOR_CICLO ON PMC_ID = OSV_PMC_ID
    LEFT JOIN ASSISTENCIA ON ASI_ID = OSV_ASI_ID
    LEFT JOIN TIPO_EVENTO ON ASI_TEV_ID = TEV_ID
    LEFT JOIN SEGMENTO ON SEG_ID = TEV_SEG_ID
    LEFT JOIN MISSAO ON MSO_OSV_ID = OSV_ID
    LEFT JOIN MISSAO_PRESTADOR ON MSP_MSO_ID = MSO_ID
    LEFT JOIN ACIONAMENTO_PRESTADOR ON APR_MSP_MSO_ID = MSO_ID
    LEFT JOIN PRESTADOR ON APR_PRS_ID = PRS_ID
    LEFT JOIN TIPO_MISSAO ON TMS_ID = MSO_TMS_ID
    INNER JOIN TIPO_ACIONAMENTO  ON MSP_TAC_ID = TAC_ID

    WHERE CONVERT(DATE, EOS_DATA_INICIO) = CONVERT(DATE, GETDATE())
    AND SOS_NOME <> 'Cancelado'
    AND APR_SMP_ID IN (2,6)
    AND EOS_TEO_ID <> 4
    AND OSV_DATA_AGENDAMENTO IS NULL
    AND SEG_ID = 1

    ORDER BY OSV_ID, EOS_TEO_ID """

    # Fazer a conexão e olhar os dados
    conexao = conectar_sql()
    df = pd.read_sql_query(queryExperiencia, conexao)
    conexao.close()

    # Criar a ordem das colunas
    experiencia = df[['Assistência','Ordem Serviço','Data abertura OS','Inicio acionamento','Data Fim','Data agendamento','Status do serviço','EOS_TEO_ID','TEO_NOME','TSE_NOME','SEG_NOME','PRS_NOME','TAC_NOME' ]]

    experienciaChegada = df[['Assistência', 'Ordem Serviço','Data abertura OS','Inicio acionamento','Data Fim','Status do serviço','TEO_NOME', 'TSE_NOME' ]]

    # Filtrar por serviços somente com status em Chegada
    experienciaChegada = experienciaChegada.loc[experienciaChegada['TEO_NOME'] == 'Chegada']


    tmp = experiencia['Inicio acionamento'] - experiencia['Data abertura OS']

    tmac = experiencia['Data Fim'] - experiencia['Inicio acionamento']

    tmc = experienciaChegada['Data Fim'] - experienciaChegada['Inicio acionamento']

    tempototal = tmp + tmac + tmc

    tempoExperiencia = tempototal.mean()

    # Crie um objeto timedelta com o valor médio
    tempo_total = pd.Series([pd.Timedelta(tempoExperiencia)])

    # Defina uma função que formata um valor timedelta no formato de hora
    def formatar_hora(delta):
        data_hora_atual = pd.to_datetime('now').replace(hour=0, minute=0, second=0, microsecond=0)
        data_hora_final = data_hora_atual + delta
        return data_hora_final.strftime('%H:%M:%S')

    # Use o método apply() para aplicar a função formatar_hora em cada valor da Series
    tempoExperienciaFinal = tempo_total.apply(formatar_hora)[0]

    return tempoExperienciaFinal



Criar os HTMLs para eu deixar o e-mail organizado.

In [ ]:
def gerar_html(tabelaAcionamento1,tabelaChegada1,resultadosAcionamento,resultadoChegada,tabelaAcionamento2,tabelaChegada2,tabelaAcionamento3,tabelaChegada3):
    ## Criar html para enviar no e-mail
    # Adiciona a tabela em HTML ao corpo do e-mail
    html = "<table border='1' cellpadding='5'>\n"
    html += "<tr><th>Assistencia</th><th>OS</th><th>dataInicio</th><th>horaInicio</th><th>duracaoAcionamento</th><th>tipoServico</th><th>Cidade</th><th>Estado</th></tr>\n"
    for i, row in tabelaAcionamento1.iterrows():
        html += "<tr>"
        html += f"<td>{row['Assistencia']}</td>"
        html += f"<td>{row['OS']}</td>"
        html += f"<td>{row['dataInicio']}</td>"
        html += f"<td>{row['horaInicio']}</td>"
        html += f"<td>{row['duracaoAcionamento']}</td>"
        html += f"<td>{row['tipoServiço']}</td>"
        html += f"<td>{row['Cidade']}</td>"
        html += f"<td>{row['Estado']}</td>"
        html += "</tr>\n"
    html += "</table>"

    # Adiciona a tabela em HTML ao corpo do e-mail
    html1 = "<table border='1' cellpadding='5'>\n"
    html1 += "<tr><th>Assistencia</th><th>OS</th><th>dataInicio</th><th>horaInicio</th><th>SLA</th><th>tempoFimPrevia</th><th>ultimaAtualizacao</th><th>tempoForaPrevia</th><th>tipoServiço</th><th>tempoExperienciaAgora</th><th>Cidade</th><th>Estado</th></tr>\n"
    for i, row in tabelaChegada1.iterrows():
        html1 += "<tr>"
        html1 += f"<td>{row['Assistencia']}</td>"
        html1 += f"<td>{row['OS']}</td>"
        html1 += f"<td>{row['dataInicio']}</td>"
        html1 += f"<td>{row['horaInicio']}</td>"
        html1 += f"<td>{row['SLA']}</td>"
        html1 += f"<td>{row['tempoFimPrevia']}</td>"
        html1 += f"<td>{row['ultimaAtualizacao']}</td>"
        html1 += f"<td>{row['tempoForaPrevia']}</td>"
        html1 += f"<td>{row['tipoServiço']}</td>"
        html1 += f"<td>{row['tempoExperienciaAgora']}</td>"
        html1 += f"<td>{row['Cidade']}</td>"
        html1 += f"<td>{row['Estado']}</td>"
        html1 += "</tr>\n"
    html1 += "</table>"

    # Vou saber a quantidade de serviços em momento de acionamento
    # Adiciona a tabela em HTML ao corpo do e-mail
    html2 = "<table border='1' cellpadding='5'>\n"
    html2 += "<tr><th>Range</th><th>Quantidade</th></tr>\n"
    for k, row in enumerate(resultadosAcionamento['Quantidade']):
        html2 += "<tr>"
        df = resultadosAcionamento.index[k]
        html2 += f"<td>{df}</td>"
        html2 += f"<td>{row}</td>"
        html2 += "</tr>\n"
    html2 += "</table>"

    # Vou saber a quantidade de serviços em momento de entrega
    # Adiciona a tabela em HTML ao corpo do e-mail
    html3 = "<table border='1' cellpadding='5'>\n"
    html3 += "<tr><th>Range</th><th>Quantidade</th></tr>\n"
    for k, row2 in enumerate(resultadoChegada['Quantidade']):
        html3 += "<tr>"
        df2 = resultadoChegada.index[k]
        html3 += f"<td>{df2}</td>"
        html3 += f"<td>{row2}</td>"
        html3 += "</tr>\n"
    html3 += "</table>"

    ## Processo de 60 à 90 minutos

    # Adiciona a tabela em HTML ao corpo do e-mail
    html4 = "<table border='1' cellpadding='5'>\n"
    html4 += "<tr><th>Assistencia</th><th>OS</th><th>dataInicio</th><th>horaInicio</th><th>duracaoAcionamento</th><th>tipoServiço</th><th>Cidade</th><th>Estado</th></tr>\n"
    for i, row in tabelaAcionamento2.iterrows():
        html4 += "<tr>"
        html4 += f"<td>{row['Assistencia']}</td>"
        html4 += f"<td>{row['OS']}</td>"
        html4 += f"<td>{row['dataInicio']}</td>"
        html4 += f"<td>{row['horaInicio']}</td>"
        html4 += f"<td>{row['duracaoAcionamento']}</td>"
        html4 += f"<td>{row['tipoServiço']}</td>"
        html4 += f"<td>{row['Cidade']}</td>"
        html4 += f"<td>{row['Estado']}</td>"
        html4 += "</tr>\n"
    html4 += "</table>"

    # Adiciona a tabela em HTML ao corpo do e-mail
    html5 = "<table border='1' cellpadding='5'>\n"
    html5 += "<tr><th>Assistencia</th><th>OS</th><th>dataInicio</th><th>horaInicio</th><th>SLA</th><th>tempoFimPrevia</th><th>ultimaAtualizacao</th><th>tempoForaPrevia</th><th>tipoServiço</th><th>tempoExperienciaAgora</th><th>Cidade</th><th>Estado</th></tr>\n"
    for i, row in tabelaChegada2.iterrows():
        html5 += "<tr>"
        html5 += f"<td>{row['Assistencia']}</td>"
        html5 += f"<td>{row['OS']}</td>"
        html5 += f"<td>{row['dataInicio']}</td>"
        html5 += f"<td>{row['horaInicio']}</td>"
        html5 += f"<td>{row['SLA']}</td>"
        html5 += f"<td>{row['tempoFimPrevia']}</td>"
        html5 += f"<td>{row['ultimaAtualizacao']}</td>"
        html5 += f"<td>{row['tempoForaPrevia']}</td>"
        html5 += f"<td>{row['tipoServiço']}</td>"
        html5 += f"<td>{row['tempoExperienciaAgora']}</td>"
        html5 += f"<td>{row['Cidade']}</td>"
        html5 += f"<td>{row['Estado']}</td>"
        html5 += "</tr>\n"
    html5 += "</table>"

    ## Processo acima de 90 minutos

    # Adiciona a tabela em HTML ao corpo do e-mail
    html6 = "<table border='1' cellpadding='5'>\n"
    html6 += "<tr><th>Assistencia</th><th>OS</th><th>dataInicio</th><th>horaInicio</th><th>duracaoAcionamento</th><th>tipoServiço</th><th>Cidade</th><th>Estado</th></tr>\n"
    for i, row in tabelaAcionamento3.iterrows():
        html6 += "<tr>"
        html6 += f"<td>{row['Assistencia']}</td>"
        html6 += f"<td>{row['OS']}</td>"
        html6 += f"<td>{row['dataInicio']}</td>"
        html6 += f"<td>{row['horaInicio']}</td>"
        html6 += f"<td>{row['duracaoAcionamento']}</td>"
        html6 += f"<td>{row['tipoServiço']}</td>"
        html6 += f"<td>{row['Cidade']}</td>"
        html6 += f"<td>{row['Estado']}</td>"
        html6 += "</tr>\n"
    html6 += "</table>"

    # Adiciona a tabela em HTML ao corpo do e-mail
    html7 = "<table border='1' cellpadding='5'>\n"
    html7 += "<tr><th>Assistencia</th><th>OS</th><th>dataInicio</th><th>horaInicio</th><th>SLA</th><th>tempoFimPrevia</th><th>ultimaAtualizacao</th><th>tempoForaPrevia</th><th>tipoServiço</th><th>tempoExperienciaAgora</th><th>Cidade</th><th>Estado</th></tr>\n"
    for i, row in tabelaChegada3.iterrows():
        html7 += "<tr>"
        html7 += f"<td>{row['Assistencia']}</td>"
        html7 += f"<td>{row['OS']}</td>"
        html7 += f"<td>{row['dataInicio']}</td>"
        html7 += f"<td>{row['horaInicio']}</td>"
        html7 += f"<td>{row['SLA']}</td>"
        html7 += f"<td>{row['tempoFimPrevia']}</td>"
        html7 += f"<td>{row['ultimaAtualizacao']}</td>"
        html7 += f"<td>{row['tempoForaPrevia']}</td>"
        html7 += f"<td>{row['tipoServiço']}</td>"
        html7 += f"<td>{row['tempoExperienciaAgora']}</td>"
        html7 += f"<td>{row['Cidade']}</td>"
        html7 += f"<td>{row['Estado']}</td>"
        html7 += "</tr>\n"
    html7 += "</table>"

    return html, html1, html2, html3, html4, html5, html6, html7

Enviar os e-mails

In [ ]:
def enviar_email(subject, html_body, remetente):
    outlook = win32.Dispatch('outlook.application')
    accounts = outlook.Session.Accounts

    if len(accounts) >= 2:
        account_segunda_opcao = accounts[1]
        mail = outlook.CreateItem(0)
        mail.SentOnBehalfOfName = account_segunda_opcao

    mail.To = remetente
    mail.Subject = subject
    mail.HTMLBody = html_body
    mail.Send()
    print(f'E-mail "{subject}" enviado com sucesso!')


Deixar tudo dentro do Loop

In [ ]:
def main_process():
    try:
        
        df, df1 = gerar_query()
        resultadoChegada, tabelaChegada1, tabelaChegada2, tabelaChegada3 = tempoChegada(df)

        resultadosAcionamento, tabelaAcionamento1, tabelaAcionamento2, tabelaAcionamento3 = tempoAcionamento(df1)

        html, html1, html2, html3, html4, html5, html6, html7 = gerar_html(tabelaAcionamento1,tabelaChegada1,resultadosAcionamento,resultadoChegada,tabelaAcionamento2,tabelaChegada2,tabelaAcionamento3,tabelaChegada3)

        tempoExperienciaFinal = tempoExperiencia()

        regex_pattern = r"Entre 30 e 60 minutos<\/td><td>(\d+)<\/td>"
        valor_tabela_2 = re.search(regex_pattern, html2).group(1)
        valor_tabela_3 = re.search(regex_pattern, html3).group(1)

        if int(valor_tabela_2) != 0 or int(valor_tabela_3) != 0:
            html_table_acionamento = tabelaAcionamento1.to_html(index=True)
            html_table_chegada = tabelaChegada1.to_html(index=True)
            html_table_monitoramento_acionamento = resultadosAcionamento.to_html(index=True)
            html_table_monitoramento_chegada = resultadoChegada.to_html(index=True)

            html_body = f""" 
            <html>
            <head>
            <title>Page Title</title>
            <style>
                table {{
                    border-collapse: collapse;
                    width: auto;
                }}
                th, td {{
                    text-align: center;
                    padding: 6px;
                }}
                th {{
                    background-color: #f2f2f2;
                }}
                tr:nth-child(even) {{
                    background-color: #f2f2f2;
                }}
            </style>
            </head>
            <body>
            <h2>REPORT SERVIÇOS EM ANDAMENTO</h2>
            <h3>SERVIÇOS EM ACIONAMENTO</h3>
            <p>Monitoramento - Consolidado</p> 
            {html_table_monitoramento_acionamento}
            <p> Monitoramento - Detalhado </p> 
            {html_table_acionamento}
            <h3>SERVIÇOS EM TEMPO DE CHEGADA</h3>
            <p>Monitoramento - Consolidado</p>
            {html_table_monitoramento_chegada}
            <p> Monitoramento - Detalhado </p>
            {html_table_chegada}
            <h2><font color="red">Tempo de Experiência do cliente - Visão dia:</font> {tempoExperienciaFinal}</h2>
            <p><b>Obs.: Somente estamos considerando serviços aceitos e sem agendamentos</b></p>
            </body>
            </html>"""

            remetente =  'claudio.bispo@tokiomarine.com.br'
            enviar_email('Serviços Auto em andamento - Até 60 minutos de atuação', html_body, remetente)
        else:
            print("Não enviar e-mail")

        # Processo para 60-90 minutos de atuação
        regex_pattern = r"Entre 60 e 90 minutos<\/td><td>(\d+)<\/td>"
        valor_tabela_2 = re.search(regex_pattern, html2).group(1)
        valor_tabela_3 = re.search(regex_pattern, html3).group(1)

        if int(valor_tabela_2) != 0 or int(valor_tabela_3) != 0:
            html_table_acionamento = tabelaAcionamento2.to_html(index=True)
            html_table_chegada = tabelaChegada2.to_html(index=True)
            html_table_monitoramento_acionamento = resultadosAcionamento.to_html(index=True)
            html_table_monitoramento_chegada = resultadoChegada.to_html(index=True)

            html_body = f""" 
            <html>
            <head>
            <title>Page Title</title>
            <style>
                table {{
                    border-collapse: collapse;
                    width: auto;
                }}
                th, td {{
                    text-align: center;
                    padding: 6px;
                }}
                th {{
                    background-color: #f2f2f2;
                }}
                tr:nth-child(even) {{
                    background-color: #f2f2f2;
                }}
            </style>
            </head>
            <body>
            <h2>REPORT SERVIÇOS EM ANDAMENTO</h2>
            <h3>SERVIÇOS EM ACIONAMENTO</h3>
            <p>Monitoramento - Consolidado</p> 
            {html_table_monitoramento_acionamento}
            <p> Monitoramento - Detalhado </p> 
            {html_table_acionamento}
            <h3>SERVIÇOS EM TEMPO DE CHEGADA</h3>
            <p>Monitoramento - Consolidado</p>
            {html_table_monitoramento_chegada}
            <p> Monitoramento - Detalhado </p>
            {html_table_chegada}
            <h2><font color="red">Tempo de Experiência do cliente - Visão dia:</font> {tempoExperienciaFinal}</h2>
            <p><b>Obs.: Somente estamos considerando serviços aceitos e sem agendamentos</b></p>
            </body>
            </html>"""

            remetente =  'claudio.bispo@tokiomarine.com.br'
            enviar_email('Serviços Auto em andamento - Até 90 minutos de atuação', html_body, remetente)
        else:
            print("Não enviar e-mail")

        # Processo para acima de 90 minutos de atuação
        regex_pattern = r"Acima de 90 minutos<\/td><td>(\d+)<\/td>"
        valor_tabela_2 = re.search(regex_pattern, html2).group(1)
        valor_tabela_3 = re.search(regex_pattern, html3).group(1)

        if int(valor_tabela_2) != 0 or int(valor_tabela_3) != 0:
            html_table_acionamento = tabelaAcionamento3.to_html(index=True)
            html_table_chegada = tabelaChegada3.to_html(index=True)
            html_table_monitoramento_acionamento = resultadosAcionamento.to_html(index=True)
            html_table_monitoramento_chegada = resultadoChegada.to_html(index=True)

            html_body = f""" 
            <html>
            <head>
            <title>Page Title</title>
            <style>
                table {{
                    border-collapse: collapse;
                    width: auto;
                }}
                th, td {{
                    text-align: center;
                    padding: 6px;
                }}
                th {{
                    background-color: #f2f2f2;
                }}
                tr:nth-child(even) {{
                    background-color: #f2f2f2;
                }}
            </style>
            </head>
            <body>
            <h2>REPORT SERVIÇOS EM ANDAMENTO</h2>
            <h3>SERVIÇOS EM ACIONAMENTO</h3>
            <p>Monitoramento - Consolidado</p> 
            {html_table_monitoramento_acionamento}
            <p> Monitoramento - Detalhado </p> 
            {html_table_acionamento}
            <h3>SERVIÇOS EM TEMPO DE CHEGADA</h3>
            <p>Monitoramento - Consolidado</p>
            {html_table_monitoramento_chegada}
            <p> Monitoramento - Detalhado </p>
            {html_table_chegada}
            <h2><font color="red">Tempo de Experiência do cliente - Visão dia:</font> {tempoExperienciaFinal}</h2>
            <p><b>Obs.: Somente estamos considerando serviços aceitos e sem agendamentos</b></p>
            </body>
            </html>"""

            remetente =  'psm.renata@tokiomarine.com.br'
            enviar_email('Serviços Auto em andamento - Acima de 90 minutos de atuação', html_body, remetente)
        else:
            print("Não enviar e-mail")


    except Exception as e:
        # Se ocorrer um erro, enviar um e-mail de relatório de erro
        erro = str(e)
        remetente = 'claudio.bispo@tokiomarine.com.br'  # Defina o e-mail de destino para relatório de erros
        enviar_email('Erro no monitoramento de serviços', f'O seguinte erro ocorreu durante o monitoramento:\n\n{erro}', remetente)

while True:
    try:
        main_process()
    except Exception as e:
        # Se ocorrer um erro no processo principal, enviar um e-mail de relatório de erro
        erro = str(e)
        remetente = 'claudio.bispo@tokiomarine.com.br'  # Defina o e-mail de destino para relatório de erros
        enviar_email('Erro no processo principal', f'O seguinte erro ocorreu durante a execução do processo principal:\n\n{erro}', remetente)

    # Esperar 60 minutos antes de executar novamente
    time.sleep(60 * 60)
